In [1]:
import torch
import torch.nn.functional as F
from pyPDMP.models import VAE
from pyPDMP.utils import loss_function

import matplotlib.pyplot as plt
import torchvision
import torchvision.datasets as datasets

In [2]:
m = VAE().cuda()

In [3]:
m

VAE(
  (fc1): Linear(in_features=2, out_features=400, bias=True)
  (fc21): Linear(in_features=400, out_features=20, bias=True)
  (fc22): Linear(in_features=400, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=400, bias=True)
  (fc4): Linear(in_features=400, out_features=2, bias=True)
)

## Training the VAE

In [4]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor()
    ])

dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

In [5]:
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=32, 
                                          shuffle=True)

In [8]:
size = 28*28

def train(m, loader, epoch):
    optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)
    m.train()
    train_loss = 0
    for batch_idx, (x, _) in enumerate(loader):
        # flatten the batch
        x = x.cuda().view(-1, size)
        optimizer.zero_grad()
        recon_batch, mu, logvar = m(x)
        loss = loss_function(recon_batch, x, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(loader.dataset),
                100. * batch_idx / len(loader),
                loss.item() / len(x)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(loader.dataset)))

In [9]:
train(m, data_loader, 10)

Train Epoch: 10 [0/60000 (0%)]	Loss: 581.390320
Train Epoch: 10 [3200/60000 (5%)]	Loss: 312.232056
Train Epoch: 10 [6400/60000 (11%)]	Loss: 297.304443
Train Epoch: 10 [9600/60000 (16%)]	Loss: 274.279114
Train Epoch: 10 [12800/60000 (21%)]	Loss: 283.327423
Train Epoch: 10 [16000/60000 (27%)]	Loss: 288.162109
Train Epoch: 10 [19200/60000 (32%)]	Loss: 277.701721
Train Epoch: 10 [22400/60000 (37%)]	Loss: 264.023682
Train Epoch: 10 [25600/60000 (43%)]	Loss: 272.127441
Train Epoch: 10 [28800/60000 (48%)]	Loss: 263.684998
Train Epoch: 10 [32000/60000 (53%)]	Loss: 278.714539
Train Epoch: 10 [35200/60000 (59%)]	Loss: 286.571350
Train Epoch: 10 [38400/60000 (64%)]	Loss: 277.598267
Train Epoch: 10 [41600/60000 (69%)]	Loss: 275.185577
Train Epoch: 10 [44800/60000 (75%)]	Loss: 258.207825
Train Epoch: 10 [48000/60000 (80%)]	Loss: 283.936157
Train Epoch: 10 [51200/60000 (85%)]	Loss: 268.934601
Train Epoch: 10 [54400/60000 (91%)]	Loss: 285.672180
Train Epoch: 10 [57600/60000 (96%)]	Loss: 280.518372
==